#Designing new proteins from a blue print
<font color='grey' size='1.5'> Created by Parisa Hosseinzadeh for *Protein Engineering and Design*, Winter 2022


In this exercise, you will get familiar with a powerful app to design and manipulate proteins.

Make sure you download this code and add it to your personal google drive.

You can read more about the application of this method in the following papers:
1. De novo design of a four-fold symmetric TIM-barrel protein with atomic-level accuracy ([link](https://www.nature.com/articles/nchembio.1966))
2. Remodel 101 from Po-Ssu Huang([link](https://sites.google.com/site/molecularengineeringhuanglab/stanford-joint-rosetta-workshop/stanford))

##Step 1. Preparing for the run

The following steps walk you through initial steps to set up everything you need for the run.

### Preparing Pyrosetta on your computer



1.   Create a folder in your google drive and name it **PyRosetta**
2.   Download `pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl` from your course files and put it in the PyRosetta folder.



In [1]:
#@title Mounting Google Drive
#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left. 


google_drive_mount_point = '/content/google_drive'

import os, sys, time

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount(google_drive_mount_point)

if not os.getenv("DEBUG"):
    google_drive = google_drive_mount_point + '/My Drive' 

Mounted at /content/google_drive


In [2]:
#@title Going to a directory of interest
#@markdown Running this cell will change the directory
#@markdown to one in your google drive.

path = '/content/google_drive/MyDrive'#@param {type:"string"}

!cd $path

In [3]:
#@title Installing PyRosetta
#@markdown Download pyrosetta. 
#@markdown This cell will take a few min to run.

%%time
if not os.getenv("DEBUG"):
    # installing PyRosetta
    if sys.version_info.major != 3 or sys.version_info.minor != 7:
        print('Need Python-3.7 to run!')
        sys.exit(1)

    # upload PyRosetta Linux WHEEL package into your google drive and put it into /PyRosetta dir
    # or alternatively you can download PyRosetta directly from GrayLab web site (but this might take some time!)
    #!mkdir $notebook_path/PyRosetta
    #!cd $notebook_path/PyRosetta && wget --user USERNAME --password PASSWORD https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python37.ubuntu.wheel/latest.html   

    pyrosetta_distr_path = google_drive + '/PyRosetta' 
    
    # finding path to wheel package, if multiple packages is found take first one
    # replace this with `wheel_path = pyrosetta_distr_path + /<wheel-file-name>.whl` if you want to use particular whl file
    wheel_path = pyrosetta_distr_path + '/' + [ f for f in os.listdir(pyrosetta_distr_path) if f.endswith('.whl')][0]
    
    print(f'Using PyRosetta wheel package: {wheel_path}')

    !pip3 install '{wheel_path}' 

Using PyRosetta wheel package: /content/google_drive/My Drive/PyRosetta/pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl
Processing ./google_drive/My Drive/PyRosetta/pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl
CPU times: user 334 ms, sys: 54.8 ms, total: 389 ms
Wall time: 40.3 s


In [4]:
#@title installing necessary modules

!pip3 install pyrosetta_help
!pip3 install Biopython

     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 
  Created wheel for pyrosetta-help: filename=pyrosetta_help-0.4.3-py3-none-any.whl size=65694 sha256=def5807802b269b9ca6c3916d43c9e240c269893a71527346ac81e233bdf7356
  Stored in directory: /root/.cache/pip/wheels/8f/8b/91/eb12fc99e62286c314b8e3ccd3bf323fd04446f4e1f9f9cb69
  Created wheel for rdkit-to-params: filename=rdkit_to_params-1.1.14-py3-none-any.whl size=45477 sha256=fc0e3a18eaa932b4b76386a1e9f90ee555f501f661b7c164281cb8e62a2b4cce
  Stored in directory: /root/.cache/pip/wheels/8d/0e/72/9a2328a7728cd177c229f2dc5470ecf6e1bfbe09a681804d5f
  Created wheel for xerox: filename=xerox-0.4.1-py3-none-any.whl size=6230 sha256=e0e2df821fabe7414f45be531276751f89d74fe96d83ea842a89690fa0ab3bb7
  Stored in directory: /root/.cache/pip/wheels/31/b1/e2/6232915cb33809221760a0db1662543bcc88ffe3276e303005
Successfully built pyrosetta-help rdkit-to-params xerox
     |███████████████████████

In [5]:
#@title Importing necessary modules

#importing modules needed from pyrosetta
from pyrosetta import init, pose_from_file
from pyrosetta.rosetta.core.pose import Pose
from pyrosetta.rosetta.protocols.forge.remodel import RemodelMover
from pyrosetta.rosetta.basic.options import *

from pyrosetta_help import Blueprinter

#required for visualization
!pip install -q py3Dmol
import py3Dmol

#import other modules
import math

import logging
logging.basicConfig(level=logging.INFO)

/usr/local/lib/python3.7/dist-packages/pyrosetta_help/ligands/load.py:11: UserWarning: rdkit_to_params couldn't be imported. LiganNicker and parameterised_pose_from_file won't work.
  warn("rdkit_to_params couldn't be imported."


##Step 2. Building new loops

In this step, we will load a pdb file (`helix_noloop.pdb`) and use remodeler to build the missing loops.

In [6]:
#@title Initializing pyrosetta

init("-mute all")

INFO:pyrosetta.rosetta:Found rosetta database at: /usr/local/lib/python3.7/dist-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [7]:
#@title Loading and visualizing the molecule
#@markdown let's take a look at the initial molecule.

pose = pose_from_file('helix_noloop.pdb')

p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open('helix_noloop.pdb','r').read(),'pdb')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.zoomTo()
p.show()

RuntimeError: ignored

In [ ]:
#@title Generating blueprint
#@markdown At the core of remodeler is the *blueprint*.
#@markdown It defines where things are, what secondary structure they have,
#@markdown and how to change them.

#@markdown Let's take a look at the current blueprint.
#@markdown **"."** means that you cannot change the residues.

blue = Blueprinter.from_pose(pose)

with open('blueprint.txt', 'w') as f:
    print(blue, file=f)

print(blue)

In [ ]:
#@title Adding loops
#@markdown Take a look at `helix_noloop.pdb` in pymol
#@markdown and find position where the cut happens at. 
#@markdown We want to add a loop in there.

#@markdown Modify the `blueprint.txt` file to add 
#@markdown a loop of size 5 and save it as `new_bl.txt`. It should look like this
#@markdown at the region you add to:

print('19 Q H')
print('0 X L')
print('0 X L')
print('0 X L')
print('0 X L')
print('0 X L')
print('20 L H')



In [ ]:
#@title Running blueprint
#@markdown Upload you file to the drive.
#@markdown Run this fast version of remodeler
#@markdown to generate the loops.

rm = RemodelMover()
set_file_option('remodel:blueprint',
                'new_bl.txt')
set_boolean_option('remodel:quick_and_dirty', 
                   True)
rm.register_options()
rm.apply(pose)


In [ ]:
#@title Checking the results
#@markdown 5 pdb molecules are generated
#@markdown after running the remodeler.
#@markdown You can download them and open them
#@markdown in pymol or check them here.

result_num =  1#@param {type:"number"}

p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open(f'{result_num}.pdb','r').read(),'pdb')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.zoomTo()
p.show()

##Step 3. Adding a new helix

In this step, we will add a new helix and loop to the original pdb.

Open the `bluepring.txt` and add a helix at the end. This is similar to the previous step but this time you're adding the new lines at the end. 

Make sure:
1. You set the type of the last residue to `H` instead of `.` .
2. You add 5 residues as a loop
3. Your helix length is the same as the other helices.
4. There should be an empty line at the end of your file.

Name this file `3_h.txt` and load it here.

In [ ]:
#@title Running blueprint
#@markdown Upload you file to the drive.
#@markdown Run this fast version of remodeler
#@markdown to generate the loops.

#@markdown **NOTE** Make sure you downloaded the
#@markdown results of last run; they will be
#@markdown overwritten.

rm = RemodelMover()
set_file_option('remodel:blueprint',
                '3_h.txt')
set_boolean_option('remodel:quick_and_dirty', 
                   True)
rm.register_options()
rm.apply(pose)


In [ ]:
#@title Checking the results
#@markdown 5 pdb molecules are generated
#@markdown after running the remodeler.
#@markdown You can download them and open them
#@markdown in pymol or check them here.

result_num =  3#@param {type:"number"}

p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open(f'{result_num}.pdb','r').read(),'pdb')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.zoomTo()
p.show()

##Step 4. Creating a new fold

Spend sometime with your groupmate and see if you can generate a new protein that has EHEE structure (sheet-helix-sheet-sheet).

Call the blueprint `ehee.txt` and run the following cells.

**NOTE:** It's best to add and remove from the original blueprint. You can easily delete sequences by removing them from the bluepring.

In [ ]:
#@title Running blueprint
#@markdown Upload you file to the drive.
#@markdown Run this fast version of remodeler
#@markdown to generate the loops.

#@markdown **NOTE:** Make sure you downloaded the
#@markdown results of last run; they will be
#@markdown overwritten.

rm = RemodelMover()
set_file_option('remodel:blueprint',
                'ehee.txt')
set_boolean_option('remodel:quick_and_dirty', 
                   True)
rm.register_options()
rm.apply(pose)


In [ ]:
#@title Checking the results
#@markdown 5 pdb molecules are generated
#@markdown after running the remodeler.
#@markdown You can download them and open them
#@markdown in pymol or check them here.

result_num =  2#@param {type:"number"}

p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open(f'{result_num}.pdb','r').read(),'pdb')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.zoomTo()
p.show()